# SageMaker Real-time Dynamic Batching Inference with Torchserve

This example demonstrates the use of dynamic batching on SageMaker with [torchserve](https://github.com/pytorch/serve/) as a model server. It demonstrates the following
1. Batch inference using DLC i.e. SageMaker's default backend container. This is done by using SageMaker python sdk in script-mode.
2. Specifying inference parameters for torchserve using environment variables.
3. Option to use a custom container with config file for torchserve baked-in the container.

**Imports**

In [1]:
! pip install --upgrade sagemaker

In [2]:
import base64
import json
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import boto3, time, json
import sagemaker

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


**Initiate session and retrieve region, account details**

In [3]:
sm_sess = sagemaker.Session()
role = sagemaker.get_execution_role()

In [4]:
sess = boto3.Session()
region = sess.region_name
account = boto3.client("sts").get_caller_identity().get("Account")

**Prepare model**

In [5]:
bucket = sm_sess.default_bucket()
prefix = "ts-dynamic-batching"
default_bucket_prefix = sm_sess.default_bucket_prefix

# If a default bucket prefix is specified, append it to the s3 path
if default_bucket_prefix:
    prefix = f"{default_bucket_prefix}/{prefix}"

model_file_name = "BERTSeqClassification"

!aws s3 cp s3://torchserve/tar_gz_files/BERTSeqClassification.tar.gz .
!aws s3 cp BERTSeqClassification.tar.gz s3://{bucket}/{prefix}/models/

f"s3://{bucket}/{prefix}/models/"

download: s3://torchserve/tar_gz_files/BERTSeqClassification.tar.gz to ./BERTSeqClassification.tar.gz
upload: ./BERTSeqClassification.tar.gz to s3://sagemaker-us-east-1-829590221516/ts-dynamic-batching/models/BERTSeqClassification.tar.gz


's3://sagemaker-us-east-1-829590221516/ts-dynamic-batching/models/'

In [6]:
model_artifact = f"s3://{bucket}/{prefix}/models/{model_file_name}.tar.gz"

In [7]:
model_name = "hf-dynamic-torchserve-sagemaker"

## Use AWS Deep Learning Container

In [8]:
# We'll use a pytorch inference DLC image that ships with sagemaker-pytorch-inference-toolkit v2.0.6. This version includes support for Torchserve environment variables used below.
image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=region,
    py_version="py39",
    image_scope="inference",
    version="1.13.1",
    instance_type="ml.c5.9xlarge",
)

In [9]:
image_uri

'763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.13.1-cpu-py39'

#### Create SageMaker model, deploy and predict

In [10]:
from sagemaker.pytorch.model import PyTorchModel

env_variables_dict = {
    "SAGEMAKER_TS_BATCH_SIZE": "3",
    "SAGEMAKER_TS_MAX_BATCH_DELAY": "100000",
    "SAGEMAKER_TS_MIN_WORKERS": "1",
    "SAGEMAKER_TS_MAX_WORKERS": "1",
}

pytorch_model = PyTorchModel(
    model_data=model_artifact,
    role=role,
    image_uri=image_uri,
    source_dir="code",
    framework_version="1.13.1",
    entry_point="inference.py",
    env=env_variables_dict,
)

In [13]:
# Change the instance type as necessary, or use 'local' for executing in Sagemaker local mode
instance_type = "ml.m4.xlarge"

predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.BytesDeserializer(),
)

[03/27/25 19:56:20] INFO     Repacking model artifact                                                  ]8;id=382382;file:///opt/conda/lib/python3.11/site-packages/sagemaker/model.py\model.py]8;;\:]8;id=44520;file:///opt/conda/lib/python3.11/site-packages/sagemaker/model.py#821\821]8;;\
                             (s3://sagemaker-us-east-1-829590221516/ts-dynamic-batching/models/BERTSeq             
                             Classification.tar.gz), script artifact (code), and dependencies ([])                 
                             into single tar.gz file located at                                                    
                             s3://sagemaker-us-east-1-829590221516/pytorch-inference-2025-03-27-19-54-             
                             40-715/model.tar.gz. This may take some time depending on model size...               

[03/27/25 19:56:49] INFO     Creating model with name: pytorch-inference-2025-03-27-19-56-49-273    ]8;id=648776;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=741765;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4094\4094]8;;\

[03/27/25 19:56:50] INFO     Creating endpoint-config with name                                     ]8;id=408106;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=93383;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#5937\5937]8;;\
                             pytorch-inference-2025-03-27-19-56-50-062                                             

                    INFO     Creating endpoint with name pytorch-inference-2025-03-27-19-56-50-062  ]8;id=62463;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=101944;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4759\4759]8;;\

---------!

## Predictions

#### By spawning a pool of 3 processes we're able to simulate requests from multiple clients and verify inference results

In [ ]:
import multiprocessing


def invoke(endpoint_name):
    predictor = sagemaker.predictor.Predictor(
        endpoint_name,
        sm_sess,
        serializer=sagemaker.serializers.JSONSerializer(),
        deserializer=sagemaker.deserializers.BytesDeserializer(),
    )
    return predictor.predict(
        "{Bloomberg has decided to publish a new report on global economic situation.}"
    )


endpoint_name = predictor.endpoint_name
pool = multiprocessing.Pool(3)
results = pool.map(invoke, 3 * [endpoint_name])
pool.close()
pool.join()
print(results)

In [ ]:
predictor.delete_endpoint(predictor.endpoint_name)

## Conclusion

Through this exercise we can see the basics of batch inference using torchserve on Amazon SageMaker. We also learnt that we can have several inference requests from different processes/users batched together, and the results will be processed as a batch of inputs. We also learnt that we could either use SageMaker's default DLC container as the base environment, and supply an inference.py script with the model, or create a custom container that can be used with SageMaker for more involved workflows.